In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
## read the files
train_data = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")


In [ ]:
## maximium length
lengths_nachor = train_data["anchor"].apply(lambda x:len(x.split(" ")))
lengths_target = train_data["target"].apply(lambda x:len(x.split(" ")))
print(max(lengths_nachor),np.argmax(lengths_target))

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
def find_common_word(words):
    
    word1 = words[0].lower()
    word2 = words[1].lower()
    common = set()
#     w1 = word1.split(" ")
    w1 = []
    w2 = []
#     w2 = word2.split(" ")
    for w in word1.split(" "):
        w1.append(ps.stem(w))
    for w in word2.split(" "):
        w2.append(ps.stem(w))
       
    common.update(w1)
    common.update(w2)
    if len(common) == len(w1)+len(w2):
        return 0
    else:
        value = len(common) - (len(w1)+len(w2))
        return abs(value)/(len(w1)+len(w2))
    
train_data["common"] = train_data[["anchor","target"]].apply(find_common_word,axis=1)

In [ ]:
train_data['modifed_score'] = train_data["score"].map(
{0. : 1,
0.25 : 2,
0.5 : 3,
0.75 : 4,
1 : 5
}
)

In [ ]:
# train_data.drop(columns=["score"],inplace=True)
y = train_data["modifed_score"]
x = train_data.drop(columns=["modifed_score","score"])


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_anchor = TfidfVectorizer()
anchor_tfid = vectorizer_anchor.fit_transform(x.anchor.values)

In [ ]:
vectorizer_target = TfidfVectorizer()
target_tfid = vectorizer_target.fit_transform(x.target.values)

In [ ]:
x.drop(columns=["anchor","target"],inplace=True)


In [ ]:
x.drop(columns=["id","context"],inplace=True)

In [ ]:
train_x = np.hstack([anchor_tfid.A,target_tfid.A,x.values])

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression()
clf.fit(train_x,y.values)
pred = clf.predict(train_x)
from sklearn.metrics import accuracy_score
accuracy_score(y.values,pred)

In [ ]:
# from sklearn.naive_bayes import MultinomialNB
# nb = MultinomialNB()
# nb.fit(train_x,y.values)
# pred =  nb.predict(train_x)
# accuracy_score(y.values,pred)

In [ ]:
## test data
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
test.head()

In [ ]:
test.drop(columns=["id","context"],inplace=True)


In [ ]:
test["common"] = test[["anchor","target"]].apply(find_common_word,axis=1)

In [ ]:
test_anchor = vectorizer_anchor.transform(test["anchor"])

In [ ]:
test_target = vectorizer_target.transform(test["target"])

In [ ]:
test_x = np.hstack([test_anchor.A,test_target.A,np.reshape(test["common"].values,(test.shape[0],1))])

In [ ]:
# nb_pred = nb.predict(test_x)

In [ ]:
ls_pred = clf.predict(test_x)

In [ ]:
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
submission = pd.DataFrame({
    "id":test.id,
    "score":ls_pred
})

In [ ]:
submission.score=submission.score.map({
    1 : 0,
     2: 0.25,
3: 0.5,
4 : 0.75,
5 : 1
})

In [ ]:
submission.to_csv("submission.csv",index=False)